# Invoke functions from module

In [1]:
import sys
sys.path.append('./modules')

In [2]:
from script_module_02a import save_property, filter_property

In [ ]:
filter_property()

# Pass MLRun context and run function from module

In [3]:
from mlrun import get_or_create_ctx
mlrun_ctx = get_or_create_ctx('mlrun_test')

> 2020-08-04 15:12:01,307 [info] logging run results to: http://mlrun-api:8080


In [4]:
source_file = "/User/igztraining/mlrun/data/demo.csv"
target_file = "/User/igztraining/mlrun/data/condos.csv" 

In [5]:
from module_handler_functions import save_property

In [6]:
save_property(mlrun_ctx,source_file=source_file,target_file=target_file)

file saved to /User/igztraining/mlrun/data/condos.csv


# Run local with MLRun

In [13]:
from mlrun import run_local

In [14]:
params = {"source_file" : "/User/igztraining/mlrun/data/demo.csv",
"target_file" : "/User/igztraining/mlrun/data/condos.csv" }

In [15]:
run_result = run_local(name='filter_property',project='Iguazio-Training',params=params,handler='filter_property',command='modules/module_handler_functions.py')

> 2020-08-04 15:24:52,944 [warning] artifact path is not defined or is local, artifacts will not be visible in the UI
> 2020-08-04 15:24:52,952 [info] starting run filter_property uid=69ae55f1341245ee91acf4ed77f68e20  -> http://mlrun-api:8080
6,5828 PEPPERMILL CT,SACRAMENTO,95841,CA,3,1,1122,Condo,Wed May 21 00:00:00 EDT 2008,89921,38.662595,-121.327813

9,11150 TRINITY RIVER DR Unit 114,RANCHO CORDOVA,95670,CA,2,2,941,Condo,Wed May 21 00:00:00 EDT 2008,94905,38.621188,-121.270555

16,7340 HAMDEN PL,SACRAMENTO,95842,CA,2,2,1134,Condo,Wed May 21 00:00:00 EDT 2008,110700,38.700051,-121.351278

18,6236 LONGFORD DR Unit 1,CITRUS HEIGHTS,95621,CA,2,1,795,Condo,Wed May 21 00:00:00 EDT 2008,116250,38.679776,-121.314089

31,5201 LAGUNA OAKS DR Unit 140,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,133000,38.423251,-121.444489

35,5201 LAGUNA OAKS DR Unit 162,ELK GROVE,95758,CA,2,2,1039,Condo,Wed May 21 00:00:00 EDT 2008,141000,38.423251,-121.444489

40,2442 LARKSPUR LN,SACRAME

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...77f68e20,0,Aug 04 15:24:53,completed,filter_property,v3io_user=adminkind=owner=adminhost=admin-jupyter-6b4b68d849-cglbr,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,,


to track results use .show() or .logs() or in CLI: 
!mlrun get run 69ae55f1341245ee91acf4ed77f68e20 --project Iguazio-Training , !mlrun logs 69ae55f1341245ee91acf4ed77f68e20 --project Iguazio-Training
> 2020-08-04 15:24:53,152 [info] run executed, status=completed


## Run local with MLRun sdk hooks

In [75]:
run_result = run_local(name='save_property',artifact_path='/User/artifacts',project='Iguazio-Training',params=params,handler='save_property',command='modules/mlrun_functions.ipynb')

> 2020-08-04 17:17:36,875 [info] starting run save_property uid=309bad2748f84b87b83e9e454d9506f2  -> http://mlrun-api:8080
/User/igztraining/mlrun/data/demo.csv


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...4d9506f2,0,Aug 04 17:17:36,completed,save_property,v3io_user=adminkind=owner=adminhost=admin-jupyter-6b4b68d849-cglbr,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run 309bad2748f84b87b83e9e454d9506f2 --project Iguazio-Training , !mlrun logs 309bad2748f84b87b83e9e454d9506f2 --project Iguazio-Training
> 2020-08-04 17:17:37,180 [info] run executed, status=completed


# Deploy code as an MLRun function

In [82]:
from mlrun import code_to_function, mlconf,mount_v3io

In [83]:
fn = code_to_function(name='save_properities', kind='job',filename='modules/mlrun_functions.ipynb')

In [84]:
fn.spec.image = "mlrun/mlrun"

In [85]:
fn.apply(mount_v3io())

In [86]:
#print(fn.to_yaml())

### Run interactive

In [87]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training')

> 2020-08-04 17:19:23,450 [info] starting run run_withcode uid=42862d7ab2c84e699e851dc69db8fa66  -> http://mlrun-api:8080
> 2020-08-04 17:19:23,623 [info] Job is running in the background, pod: run-withcode-nhlp2
/User/igztraining/mlrun/data/demo.csv
> 2020-08-04 17:19:28,130 [info] run executed, status=completed
final state: succeeded


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
Iguazio-Training,...9db8fa66,0,Aug 04 17:19:27,completed,run_withcode,v3io_user=adminkind=jobowner=adminhost=run-withcode-nhlp2,,source_file=/User/igztraining/mlrun/data/demo.csvtarget_file=/User/igztraining/mlrun/data/condos.csv,properties_available=54,condo_properties


to track results use .show() or .logs() or in CLI: 
!mlrun get run 42862d7ab2c84e699e851dc69db8fa66 --project Iguazio-Training , !mlrun logs 42862d7ab2c84e699e851dc69db8fa66 --project Iguazio-Training
> 2020-08-04 17:19:29,766 [info] run executed, status=completed


In [88]:
sp_run.outputs

{'properties_available': 54,
 'condo_properties': 'store://Iguazio-Training/run_withcode_condo_properties#42862d7ab2c84e699e851dc69db8fa66'}

### Run and forget

In [ ]:
sp_run = fn.run(name='run_withcode',artifact_path="/User/artifacts",handler='save_property',params=params,project='Iguazio-Training',watch=False)